In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
os.environ["KAGGLE_USERNAME"] = "magdhndi"
os.environ["KAGGLE_KEY"] = "9b6c8953fb75d807a407f863ae22edc6"

In [ ]:
!kaggle datasets download tawsifurrahman/covid19-radiography-database
!kaggle datasets download artyomkolas/3-kinds-of-pneumonia
!kaggle datasets download darshan1504/covid19-detection-xray-dataset

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
 98% 764M/778M [00:08<00:00, 88.6MB/s]
100% 778M/778M [00:08<00:00, 101MB/s] 
Dataset URL: https://www.kaggle.com/datasets/artyomkolas/3-kinds-of-pneumonia
License(s): Attribution 4.0 International (CC BY 4.0)
100% 3.48G/3.49G [00:41<00:00, 86.6MB/s]
100% 3.49G/3.49G [00:41<00:00, 90.0MB/s]
Dataset URL: https://www.kaggle.com/datasets/darshan1504/covid19-detection-xray-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 95% 177M/186M [00:02<00:00, 60.0MB/s]
100% 186M/186M [00:02<00:00, 77.0MB/s]


In [ ]:
!unzip covid19-radiography-database
!unzip 3-kinds-of-pneumonia
!unzip covid19-detection-xray-dataset

In [ ]:
normal_paths = ['COVID-19_Radiography_Dataset/Normal/images', 'Curated X-Ray Dataset/Normal', 'NonAugmentedTrain/Normal']
covid_paths = ['COVID-19_Radiography_Dataset/COVID/images', 'Curated X-Ray Dataset/COVID-19', 'NonAugmentedTrain/COVID-19']
viralPneumonia_paths = ['COVID-19_Radiography_Dataset/Viral Pneumonia/images', 'Curated X-Ray Dataset/Pneumonia-Viral', 'NonAugmentedTrain/ViralPneumonia']
Lung_Opacity_paths = ['COVID-19_Radiography_Dataset/Lung_Opacity/images']
BacterialPneumonia_paths = ['Curated X-Ray Dataset/Pneumonia-Bacterial', 'NonAugmentedTrain/BacterialPneumonia']

In [ ]:
all_paths = [normal_paths, covid_paths, viralPneumonia_paths, BacterialPneumonia_paths, Lung_Opacity_paths]

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images(paths, target_label, target_size=(100, 100), Max=3413):
    images = []
    labels = []

    for folder_path in paths:
        images_names = os.listdir(folder_path)
        for image_name in images_names:
          if len(images) >= Max:
            break
          image_path = os.path.join(folder_path, image_name)
          image = load_img(image_path, target_size=target_size)
          image_array = img_to_array(image) / 255.0  # Normalize the image
          images.append(image_array)
          labels.append(target_label)  # Use the corresponding label from 'target' list

    images = np.array(images)
    labels = np.array(labels)

    return images, labels


In [ ]:
# Preprocess image data
data_images = []
data_targets = []
for target_label, db_images in enumerate(all_paths):
  images, target = load_images(db_images, target_label)
  data_images.extend(images)
  data_targets.extend(target)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def encode_labels(labels):
    """
    Encode categorical labels into a numerical format using one-hot encoding.

    Args:
        labels (np.array): Array of categorical labels.

    Returns:
        np.array: One-hot encoded labels.
    """
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    one_hot_encoded = to_categorical(integer_encoded)

    return one_hot_encoded, label_encoder


In [ ]:
# Encode labels
labels_one_hot, label_encoder = encode_labels(data_targets)

In [ ]:
from sklearn.model_selection import train_test_split

def split_dataset(images, labels, test_size=0.2, val_size=0.1, random_state=42):
    """
    Split dataset into training, validation, and test sets.

    Args:
        images (np.array): Array of images.
        labels (np.array): Array of labels.
        test_size (float): Proportion of the data to include in the test split.
        val_size (float): Proportion of the data to include in the validation split.
        random_state (int): Seed used by the random number generator.

    Returns:
        tuple: Split data (X_train, X_val, X_test, y_train, y_val, y_test).
    """
    X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=(test_size + val_size), random_state=random_state)
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_ratio, random_state=random_state)

    return np.asarray(X_train), np.asarray(X_val), np.asarray(X_test), np.asarray(y_train), np.asarray(y_val), np.asarray(y_test)


In [ ]:
# Split dataset
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(data_images, labels_one_hot)

In [ ]:
data_images, labels_one_hot = 0, 0

In [ ]:
def shuffle_data(X, y):
    """
    Shuffle the data to ensure randomness.

    Args:
        X (np.array): Array of images.
        y (np.array): Array of labels.

    Returns:
        tuple: Shuffled data (X_shuffled, y_shuffled).
    """
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    return X[indices], y[indices]

In [ ]:
# Shuffle data
X_train, y_train = shuffle_data(X_train, y_train)
X_val, y_val = shuffle_data(X_val, y_val)
X_test, y_test = shuffle_data(X_test, y_test)

In [ ]:
len(X_train), len(y_train)

(11945, 11945)

In [ ]:
len(X_val), len(y_val)

(3413, 3413)

In [ ]:
len(X_test), len(y_test)

(1707, 1707)

In [ ]:
import tensorflow as tf

def prepare_batches(X, y, batch_size=32):
    """
    Prepare data batches for training.

    Args:
        X (np.array): Array of images.
        y (np.array): Array of labels.
        batch_size (int): Size of the batches.

    Returns:
        tf.data.Dataset: Batched dataset.
    """
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.batch(batch_size)
    return dataset


In [ ]:
# Prepare batches
train_dataset = prepare_batches(X_train, y_train)
val_dataset = prepare_batches(X_val, y_val)
test_dataset = prepare_batches(X_test, y_test)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = 0,0,0, 0,0,0

In [ ]:
def cache_and_prefetch(dataset, cache=True, buffer_size=tf.data.AUTOTUNE):
    """
    Cache and prefetch the dataset for performance optimization.

    Args:
        dataset (tf.data.Dataset): Input dataset.
        cache (bool): Whether to cache the dataset.
        buffer_size (int): Buffer size for prefetching.

    Returns:
        tf.data.Dataset: Optimized dataset.
    """
    if cache:
        dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size)
    return dataset


In [ ]:
# Cache and prefetch
train_dataset = cache_and_prefetch(train_dataset)
val_dataset = cache_and_prefetch(val_dataset)
test_dataset = cache_and_prefetch(test_dataset, cache=False)  # Typically don't cache test set

In [ ]:
len(train_dataset)

374

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the CNN model with regularization
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
predictions = Dense(5, activation='softmax')(x)

image_model = Model(inputs=base_model.input, outputs=predictions)
image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)


In [ ]:
# Class weights to balance the contributions of each class
class_weights = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}


In [ ]:
# Number of epochs for initial training
initial_epochs = 10

# Train the model
history = image_model.fit(train_dataset,
                          validation_data=val_dataset,
                          epochs=initial_epochs,
                          class_weight=class_weights,
                          callbacks=[early_stopping, model_checkpoint])


Epoch 1/10
374/374 [==============================] - ETA: 0s - loss: 6.0677 - accuracy: 0.7378

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


374/374 [==============================] - 86s 124ms/step - loss: 6.0677 - accuracy: 0.7378 - val_loss: 2.0516 - val_accuracy: 0.2543
Epoch 2/10
374/374 [==============================] - 42s 112ms/step - loss: 0.7224 - accuracy: 0.7890 - val_loss: 1.3593 - val_accuracy: 0.4216
Epoch 3/10
374/374 [==============================] - 44s 118ms/step - loss: 0.6224 - accuracy: 0.8148 - val_loss: 1.1734 - val_accuracy: 0.5640
Epoch 4/10
374/374 [==============================] - 46s 124ms/step - loss: 0.5701 - accuracy: 0.8366 - val_loss: 1.0261 - val_accuracy: 0.5734
Epoch 5/10
374/374 [==============================] - 40s 108ms/step - loss: 0.5622 - accuracy: 0.8499 - val_loss: 1.1039 - val_accuracy: 0.6408
Epoch 6/10
374/374 [==============================] - 41s 109ms/step - loss: 0.5071 - accuracy: 0.8669 - val_loss: 1.7513 - val_accuracy: 0.4629
Epoch 7/10
374/374 [==============================] - 49s 130ms/step - loss: 0.4203 - accuracy: 0.8891 - val_loss: 0.7140 - val_accuracy: 0.7

In [ ]:
# Unfreeze the top layers of the base model
for layer in base_model.layers[-20:]:  # Adjusted number of layers to unfreeze
    layer.trainable = True


In [ ]:
# Recompile the model with a lower learning rate for fine-tuning
image_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


In [ ]:
# Number of epochs for fine-tuning
fine_tuning_epochs = 20  # Adjusted based on expected complexity

# Continue training the model
fine_tuning_history = image_model.fit(train_dataset,
                                      validation_data=val_dataset,
                                      epochs=fine_tuning_epochs,
                                      class_weight=class_weights,
                                      callbacks=[early_stopping, model_checkpoint])


Epoch 1/20
374/374 [==============================] - 27s 53ms/step - loss: 0.4670 - accuracy: 0.8625 - val_loss: 0.5382 - val_accuracy: 0.8213
Epoch 2/20
374/374 [==============================] - 19s 50ms/step - loss: 0.3667 - accuracy: 0.8872 - val_loss: 0.4889 - val_accuracy: 0.8330
Epoch 3/20
374/374 [==============================] - 19s 51ms/step - loss: 0.3267 - accuracy: 0.8965 - val_loss: 0.4650 - val_accuracy: 0.8412
Epoch 4/20
374/374 [==============================] - 19s 51ms/step - loss: 0.2931 - accuracy: 0.9066 - val_loss: 0.4534 - val_accuracy: 0.8476
Epoch 5/20
374/374 [==============================] - 19s 51ms/step - loss: 0.2583 - accuracy: 0.9206 - val_loss: 0.4470 - val_accuracy: 0.8573
Epoch 6/20
374/374 [==============================] - 18s 49ms/step - loss: 0.2257 - accuracy: 0.9321 - val_loss: 0.4472 - val_accuracy: 0.8640
Epoch 7/20
374/374 [==============================] - 18s 49ms/step - loss: 0.1965 - accuracy: 0.9416 - val_loss: 0.4512 - val_accuracy:

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = image_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

54/54 [==============================] - 3s 44ms/step - loss: 0.4410 - accuracy: 0.8612
Test accuracy: 86.12%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_model.save('/content/drive/My Drive/path_to_save/model.h5')

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def TFNP(cm):
    """
    Calculate True Positives, False Negatives, False Positives, and True Negatives for each class.
    """
    all_samples = np.sum(cm)
    cm_classes = []

    for i in range(len(cm)):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = all_samples - tp - fp - fn
        cm_classes.append([[tp, fn], [fp, tn]])

    return np.asarray(cm_classes)


In [ ]:
def Report(cm_classes):
    """
    Generate performance metrics report for each class.
    """
    repo = []
    for i in range(len(cm_classes)):
        tp = cm_classes[i, 0, 0]
        fn = cm_classes[i, 0, 1]
        fp = cm_classes[i, 1, 0]
        tn = cm_classes[i, 1, 1]

        iou = tp / (tp + fn + fp)  # Intersection over Union
        dsc = (2 * tp) / ((2 * tp) + fp + fn)  # Dice Similarity Coefficient
        acc = (tp + tn) / (tp + tn + fp + fn)  # Accuracy
        precision = tp / (tp + fp)  # Precision (Positive Predictive Value)
        recall = tp / (tp + fn)  # Recall (Sensitivity)
        spec = tn / (tn + fp)  # Specificity
        f1_score = 2 * (precision * recall) / (precision + recall)  # F1-Score

        info = {
            'IOU': iou,
            'DSC': dsc,
            'ACC': acc,
            'Specificity': spec,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        }

        repo.append(info)

    return np.asarray(repo)


In [ ]:
def model_performance_report(model, test_dataset, class_labels):
    """
    Generate a performance report for a given model and test data.
    """
    y_pred_all = []
    y_test_all = []

    for x_test, y_test in test_dataset:
        y_pred = model.predict(x_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test, axis=1)

        y_pred_all.extend(y_pred_classes)
        y_test_all.extend(y_test_classes)

    y_pred_all = np.array(y_pred_all)
    y_test_all = np.array(y_test_all)

    cm = confusion_matrix(y_test_all, y_pred_all)
    cm_classes = TFNP(cm)
    report = Report(cm_classes)

    for i, label in enumerate(class_labels):
        print(f"The report for {label} is : \n {report[i]} \n")

    return report, cm



In [ ]:
# Example usage:
# Assuming you have a trained model `image_model`, test dataset `test_dataset`, and class labels `disease_labels`
disease_labels = ["Normal", "Covid-19", "Viral Pneumonia", "Bacterial Pneumonia", "Lung Opacity"]
report, cm = model_performance_report(best_model_v2, test_dataset, disease_labels)

In [ ]:
cm

array([[288,   6,  10,   4,  19],
       [ 10, 315,   0,   0,  12],
       [  0,   2, 313,  49,   0],
       [  0,   2,  81, 247,   0],
       [ 30,  10,   1,   1, 307]])

In [ ]:
disease = {0:"Normal", 1:"Covid-19", 2:"Viral Pneumonia", 3:"Bacterial Pneumonia", 4:"Lung Opacity"}
for i in range(len(report)):
  print(f"The report for {disease[i]} is : \n {report[i]} \n")

The report for Normal is : 
 {'IOU': 0.784741144414169, 'DSC': 0.8793893129770992, 'ACC': 0.9537199765670767, 'Specificity': 0.9710144927536232, 'Precision': 0.8780487804878049, 'Recall': 0.8807339449541285, 'F1-Score': 0.8793893129770993} 

The report for Covid-19 is : 
 {'IOU': 0.8823529411764706, 'DSC': 0.9375, 'ACC': 0.9753954305799648, 'Specificity': 0.9854014598540146, 'Precision': 0.9402985074626866, 'Recall': 0.9347181008902077, 'F1-Score': 0.9375} 

The report for Viral Pneumonia is : 
 {'IOU': 0.6864035087719298, 'DSC': 0.8140442132639792, 'ACC': 0.9162272993555947, 'Specificity': 0.9314966492926284, 'Precision': 0.7728395061728395, 'Recall': 0.8598901098901099, 'F1-Score': 0.8140442132639792} 

The report for Bacterial Pneumonia is : 
 {'IOU': 0.6432291666666666, 'DSC': 0.7828843106180665, 'ACC': 0.9197422378441711, 'Specificity': 0.9607843137254902, 'Precision': 0.8205980066445183, 'Recall': 0.7484848484848485, 'F1-Score': 0.7828843106180666} 

The report for Lung Opacity i